## Scaling Strategies (Caching)

Caching is a crucial optimization technique for production systems. It can significantly reduce latency and computational load by storing frequently accessed results.

### Why Implement Caching?

1. **Reduced Latency** - Cached results can be returned instantly without computing embeddings or searching the vector space
2. **Lower Computational Costs** - Fewer embedding calculations mean lower GPU/CPU usage
3. **Better Scalability** - Handle more queries with the same resources

We'll implement a simple LRU (Least Recently Used) cache and measure its performance impact.

# Setup and Initialization

In [1]:
# Install the required packages
!uv pip install accelerate==1.6.0 sentence-transformers==4.0.2

Using Python 3.12.9 environment at: /workspaces/fundamentals-of-ai-engineering-principles-and-practical-applications-6026542/.venv
Audited 2 packages in 9ms


In [2]:
# Import necessary libraries
import chromadb
from chromadb.utils import embedding_functions
import time
import random

### LRU Cache Implementation

Let's implement a simple LRU (Least Recently Used) cache. This type of cache keeps track of which queries are used most frequently and evicts the least recently used entries when the cache is full.

In [3]:
print("\n=== CACHING IMPLEMENTATION ===")

# Implement a simple LRU cache
class LRUCache:
    def __init__(self, capacity=100):
        self.capacity = capacity  # Maximum number of items the cache can hold
        self.cache = {}           # Dictionary to store cache items
        self.usage_order = []     # List to track access order
    
    def get(self, key):
        if key in self.cache:
            # Update usage order - move to end of list (most recently used)
            self.usage_order.remove(key)
            self.usage_order.append(key)
            return self.cache[key]
        return None  # Cache miss
    
    def put(self, key, value):
        if key in self.cache:
            # Update existing entry
            self.cache[key] = value
            self.usage_order.remove(key)
            self.usage_order.append(key)
        else:
            # Add new entry
            if len(self.cache) >= self.capacity:
                # Evict least recently used item
                lru_key = self.usage_order.pop(0)
                del self.cache[lru_key]
            
            self.cache[key] = value
            self.usage_order.append(key)
    
    def clear(self):
        self.cache = {}
        self.usage_order = []
    
    def __len__(self):
        return len(self.cache)


=== CACHING IMPLEMENTATION ===


### Setting Up the Collection

Now let's create a collection and populate it with sample documents for our caching experiment.

In [4]:
# Initialize Chroma
client = chromadb.Client()
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# Create a collection
collection = client.create_collection(
    name="cache_test",
    embedding_function=embedding_function
)

In [5]:
# Add sample documents
num_docs = 1000
documents = [
    f"This is a sample document {i} with various content for testing caching" for i in range(num_docs)]
ids = [f"cache_doc_{i}" for i in range(num_docs)]

# Add documents in batches to avoid overwhelming the system
for i in range(0, num_docs, 100):
    end_idx = min(i + 100, num_docs)

    collection.add(
        documents=documents[i:end_idx],
        ids=ids[i:end_idx]
    )

print(f"Added {num_docs} documents to the collection")

Added 1000 documents to the collection


### Cached Query Function

Let's implement a function that uses our cache to store and retrieve query results.

In [6]:
# Initialize cache with a capacity of 50 entries
query_cache = LRUCache(capacity=50)

# Function to query with caching
def cached_query(query_text, n_results=10, use_cache=True):
    # Create a unique cache key from the query text and number of results
    cache_key = f"{query_text}:{n_results}"

    if use_cache:
        # Check cache first
        cached_result = query_cache.get(cache_key)
        if cached_result is not None:
            return cached_result, True  # Cache hit

    # Cache miss or cache disabled, perform actual query
    result = collection.query(
        query_texts=[query_text],
        n_results=n_results
    )

    if use_cache:
        # Update cache
        query_cache.put(cache_key, result)

    return result, False  # Cache miss

### Preparing Query Mix

To simulate a realistic workload, we'll create a mix of common (frequently repeated) and unique queries.

In [7]:
# Test queries with varying cache hit rates
print("\nTesting query performance with caching:")

# Prepare query mix (some repeated, some unique)
common_queries = [
    "document with content",
    "sample document",
    "testing caching",
    "various content"
]

unique_queries = [f"unique query {i}" for i in range(50)]

# Mix queries with different distributions to test cache performance
mixed_queries = []
for _ in range(20):
    # Add common queries (higher probability)
    mixed_queries.extend(common_queries)
    
    # Add some unique queries
    mixed_queries.extend(random.sample(unique_queries, 5))

# Shuffle to ensure realistic query pattern
random.shuffle(mixed_queries)


Testing query performance with caching:


### Benchmark: No Cache vs. With Cache

Now let's measure the performance difference between running queries without a cache versus with a cache.

In [8]:
# Run without cache
print("Running queries without cache...")
start_time = time.time()

for query in mixed_queries:
    _, _ = cached_query(query, use_cache=False)

no_cache_time = time.time() - start_time

Running queries without cache...


In [9]:
# Run with cache
print("Running queries with cache...")
query_cache.clear()  # Clear the cache

start_time = time.time()
hits = 0

for query in mixed_queries:
    _, is_hit = cached_query(query, use_cache=True)
    if is_hit:
        hits += 1

with_cache_time = time.time() - start_time
hit_rate = hits / len(mixed_queries)

Running queries with cache...


In [10]:
# Report results
print("\nCache Performance Results:")
print(f"  Without cache: {no_cache_time:.4f} seconds")
print(f"  With cache: {with_cache_time:.4f} seconds")
print(
    f"  Time saved: {no_cache_time - with_cache_time:.4f} seconds ({(1 - with_cache_time/no_cache_time) * 100:.1f}%)")
print(f"  Cache hit rate: {hit_rate:.1%}")
print(f"  Cache size: {len(query_cache)}")


Cache Performance Results:
  Without cache: 2.1609 seconds
  With cache: 0.5848 seconds
  Time saved: 1.5761 seconds (72.9%)
  Cache hit rate: 72.8%
  Cache size: 49


## Advanced Scaling Strategies

### Horizontal Scaling Approaches

As your vector database grows beyond the capacity of a single machine, you'll need to implement horizontal scaling strategies. Here are some common approaches:

1. **Sharding** - Partitioning your vector space across multiple instances
   - **By ID range** - Deterministic but may lead to unbalanced shards
   - **By vector clustering** - Better search performance but more complex

2. **Replication** - Creating copies of your data across multiple instances
   - Improves read throughput and fault tolerance
   - Requires synchronization mechanisms for writes

3. **Hybrid approaches** - Combining sharding and replication
   - Example: ChromaDB cluster with data sharded across nodes and each shard replicated

### Resource Management Best Practices

1. **Memory Optimization**
   - Use quantization to reduce vector size (e.g., 32-bit to 8-bit)
   - Implement disk-based storage for less frequently accessed vectors

2. **CPU Utilization**
   - Batch similar operations
   - Use asynchronous processing where possible

3. **Network Efficiency**
   - Minimize data transfer between components
   - Compress payloads when possible

### Real-world Implementation Considerations

1. **Monitoring and Observability**
   - Track latency, throughput, and error rates
   - Set up alerts for performance degradation

2. **Failure Handling**
   - Implement graceful degradation strategies
   - Consider fallback search methods

3. **Update Strategies**
   - Batch updates to reduce index rebuilding frequency
   - Consider incremental index updates

4. **Hybrid Search Approaches**
   - Combine vector search with keyword search for better results
   - Filter vectors based on metadata before computing distances

## Conclusion and Key Takeaways

In this notebook, we've explored practical approaches to scaling vector databases for production use:

**Caching**
   - Can significantly reduce latency (70%+ in our example)
   - Most effective when query patterns show temporal locality

**Advanced Scaling Strategies**
   - Horizontal scaling through sharding and replication
   - Resource optimization across memory, CPU, and network
   - Operational considerations for production deployments